In [1]:
import pymysql
import pandas as pd

In [ ]:
# 라이브러리 설치
# !pip install python-dotenv

In [4]:
# .env 파일에서 변수를 가지고 오기 위한 라이브러리 로드
import os
from dotenv import load_dotenv

In [5]:
# .env 파일을 환경변수로 지정: load_dotenv()
# 현재 작업 경로에서 .env 파일을 찾아서 안에 존재하는 변수들을 환경변수로 지정
load_dotenv()

True

In [12]:
# .env에서 설정한 변수들을 로드
os.getenv('host')

'127.0.0.1'

In [14]:
# DB server와 연결
_db = pymysql.connect(
    host = os.getenv('host'),   # '127.0.0.1'
    port = int(os.getenv('port')),   # 3306
    user = os.getenv('user'),   # 'root'
    password = os.getenv('pw'),   # '1234'
    db = os.getenv('db_name')   # 'ubion'
)

In [ ]:
# # DB server와 연결 -> 기존 방법
# _db = pymysql.connect(
#     host = '127.0.0.1',
#     port = 3306,
#     user = 'root',
#     password = '1234',
#     db = 'ubion'
# )

In [15]:
# cursor라는 공간을 생성
cursor = _db.cursor(pymysql.cursors.DictCursor)

#### - CRUD
- CUD (insert, update, delete)
    1. query문 생성
    2. execute() 함수를 이용해 cursor에 명령어 전달
    3. DB server에 commit() 함수를 이용해 cursor와 DB server에 동기화
    4. DB server와의 연결 종료 close()
- R (select)
    1. query문 생성
    2. execute() 함수를 이용해 cursor에 명령어 전달
    3. fetchall()을 함수를 이용해 cursor에서 select 명령어를 이용해 데이터를 가지고 옴
    4. 3번 과정에서 불러온 데이터를 데이터프레임으로 변환
    5. close() 함수를 이용해 DB server와의 연결 종료

In [ ]:
# user_info 테이블에 데이터 대입
insert_query= '''
    INSERT INTO
    `user_info`
    VALUES
    (%s, %s, %s, %s)
'''
data_list = ['test99', '1111', 'kim', 50]
# 쿼리문에 %s가 존재한다면 두번째 인자값에 있는 데이터들을 하나씩 대입
# 쿼리문의 %s의 개수와 리스트의 데이터 개수는 같아야 함
cursor.execute(insert_query, data_list)


1

In [ ]:
select_query = '''
    SELECT * FROM `user_info`
'''
data_list = []
cursor.execute(select_query, data_list)

5

In [18]:
cursor.fetchall()

[{'id': 'test', 'password': '1234', 'name': 'kim', 'age': 30},
 {'id': 'test10', 'password': '0000', 'name': 'lee', 'age': 30},
 {'id': 'test2', 'password': '0000', 'name': 'park', 'age': 20},
 {'id': 'test3', 'password': '9999', 'name': 'lee', 'age': 50},
 {'id': 'test99', 'password': '1111', 'name': 'kim', 'age': 50}]

In [22]:
# CUD 쿼리문을 사용할 때의 함수 생성
def cud_query(_query, _data_list = []):
    # _query -> CUD(insert, update, delete)로 이루어진 sql query문
    # _data_list -> _query에 %s format이 존재하면 데이터들을 받아오기 위한 공간
        # 기본값=[] 빈 리스트
    # execute() 함수를 이용해 cursor에 query 전송
    try:
        # execute() 함수의 호출에 문제가 없다면
        cursor.execute(_query, _data_list)
        print('Query OK!')
    except:
        # execute() 함수의 호출에 문제 발생 시
        print('Query Error!')

In [23]:
# R 쿼리문 함수 생성
def r_query(_query, *_data_list):
    # execute() 함수 호출
    try:
        cursor.execute(_query, _data_list)
        # execute() 결과를 변수에 대입
        result = cursor.fetchall()
        # 결과 반환
        return result
    except:
        print('Query Error!')

In [24]:
# insert 쿼리문을 생성해 cud_query() 함수 호출
query1 = '''
    INSERT INTO
    `user_info`
    VALUES
    (%s, %s, %s, %s)
'''
data_list = ['test15', '1515', 'choi', 20]

cud_query(query1, data_list)

Query OK!


In [31]:
# select 쿼리문 생성해 r_query() 함수 호출
query2 = '''
    SELECT * FROM `user_info`
    WHERE
    `id` in (%s, %s)
'''

r_query(query2, 'test99', 'test15')

[{'id': 'test15', 'password': '1515', 'name': 'choi', 'age': 20},
 {'id': 'test99', 'password': '1111', 'name': 'kim', 'age': 50}]

In [34]:
query2.startswith('SELECT')

False

In [ ]:
query2      # 공백 존재

'\n    SELECT * FROM `user_info`\n    WHERE\n    `id` in (%s, %s)\n'

In [ ]:
# lstrip() -> 공백 제거, lower() -> 소문자로 변경(대소문자 통일)
query2.lstrip().lower().startswith('select')

False

In [45]:
# split() -> 공백으로 문자 분리
query2.split()[0] in ('select', 'SELECT')

True

In [46]:
# lower(), upper() 추천 -> 대소문자가 섞여있는 경우도 있어
query2.split()[0].lower() == 'select'

True

In [47]:
# CRUD를 모두 사용할 수 있는 함수 생성
def sql_query(_query, *_data_list):
    try:
        cursor.execute(_query, _data_list)
        # _query가 select인가?
        # if _query.lstrip().lower().startswith('select'):
        if _query.lower().split()[0] == 'select':
            result = cursor.fetchall()
            return result
        else:
            print('Query OK!')
    except Exception as e:
        print('Query Error')
        print(e)

In [49]:
query3 = '''
    SELECT * from `user_info`
'''

sql_query(query3)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'age': 30},
 {'id': 'test10', 'password': '0000', 'name': 'lee', 'age': 30},
 {'id': 'test15', 'password': '1515', 'name': 'choi', 'age': 20},
 {'id': 'test2', 'password': '0000', 'name': 'park', 'age': 20},
 {'id': 'test3', 'password': '9999', 'name': 'lee', 'age': 50},
 {'id': 'test99', 'password': '1111', 'name': 'kim', 'age': 50}]

In [50]:
query4 = '''
    delete from `user_info`
    where `id` = %s
'''
sql_query(query4, 'test15')

Query OK!


In [54]:
sql_query(query4, 'test14')

Query OK!


In [55]:
sql_query(query3)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'age': 30},
 {'id': 'test10', 'password': '0000', 'name': 'lee', 'age': 30},
 {'id': 'test2', 'password': '0000', 'name': 'park', 'age': 20},
 {'id': 'test3', 'password': '9999', 'name': 'lee', 'age': 50},
 {'id': 'test99', 'password': '1111', 'name': 'kim', 'age': 50}]

In [56]:
_db.commit()

In [ ]:
# DB server와의 연결 종료
_db.close()

In [ ]:
# mysql과 연동하고 sql 쿼리문을 보내고 결과 받아오는 class 생성
class MyDB:
    # 생성자 함수
    # 매개변수: 서버의 정보(기본값 -> 내 컴퓨터의 DB 정보)
    def __init__(
            self,
            _host = '127.0.0.1',
            _port = 3306,
            _user = 'root',
            _pw = '1234',
            _db_name = 'ubion'
    ):
        # 생성된 class에서 독립적으로 사용하려는 변수를 등록하는 과정
        self.host = _host
        self.port = _port
        self.user = _user
        self.pw = _pw
        self.db = _db_name
        
    # DB 서버와 연결, cursor 생성, query문 전송 후 필요에 따라 결과값 리턴하는 함수
    def sql_query(self, _query, *_data_list):
        # 서버와의 연결
        self._db = pymysql.connect(     
            host = self.host,
            port = self.port,
            user = self.user,
            password = self.pw,
            db = self.db
        )

        # cursor 생성
        cursor = self._db.cursor(pymysql.cursors.DictCursor)
        cursor.execute(_query, _data_list)
        
        # _query가 select문이라면
        # if _query.lower().split()[0] == 'select':
        if _query.upper().lstrip().startswith('SELECT'):
            result = cursor.fetchall()
            return result
        
        else:
            print('Query OK!')
    
    def commit_db(self):
        self._db.commit()
        print('커밋 완료')
        self._db.close()
        print('Close 완료')

In [86]:
# class 생성
db1 = MyDB()

In [87]:
select_query = '''
    select * from `user_info`
'''

db1.sql_query(select_query)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'age': 30},
 {'id': 'test10', 'password': '0000', 'name': 'lee', 'age': 30},
 {'id': 'test2', 'password': '0000', 'name': 'park', 'age': 20},
 {'id': 'test3', 'password': '9999', 'name': 'lee', 'age': 50},
 {'id': 'test99', 'password': '1111', 'name': 'kim', 'age': 50}]

In [92]:
update_query = '''
    update
    `user_info`
    set
    `password` = %s
    where
    `id` = %s
'''

db1.sql_query(update_query, '0123', 'test99')

Query OK!


In [93]:
db1.commit_db()

커밋 완료
Close 완료


In [94]:
db1.sql_query(select_query)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'age': 30},
 {'id': 'test10', 'password': '0000', 'name': 'lee', 'age': 30},
 {'id': 'test2', 'password': '0000', 'name': 'park', 'age': 20},
 {'id': 'test3', 'password': '9999', 'name': 'lee', 'age': 50},
 {'id': 'test99', 'password': '0123', 'name': 'kim', 'age': 50}]

In [95]:
# MyDB class 생성해 외부의 DB server에 접속
db2 = MyDB(
    _host = '172.30.1.187',
    _port = 3306,
    _user = 'ubion',
    _pw = '1234',
    _db_name = 'ubion'
)

In [ ]:
# db1 -> 내 컴퓨터 DB
# db2 -> 외부의 컴퓨터 DB
select_aapl = '''
    select * from aapl
'''

In [ ]:
db1.sql_query(select_aapl)      # 에러 발생 -> 내 컴퓨터에는는 존재하지 않는 테이블

ProgrammingError: (1146, "Table 'ubion.aapl' doesn't exist")

In [98]:
db2.sql_query(select_aapl)

[{'Date': '1980-12-12',
  'Open': 0.513393,
  'High': 0.515625,
  'Low': 0.513393,
  'Close': 0.513393,
  'Adj Close': 0.410525,
  'Volume': 117258400},
 {'Date': '1980-12-15',
  'Open': 0.488839,
  'High': 0.488839,
  'Low': 0.486607,
  'Close': 0.486607,
  'Adj Close': 0.389106,
  'Volume': 43971200},
 {'Date': '1980-12-16',
  'Open': 0.453125,
  'High': 0.453125,
  'Low': 0.450893,
  'Close': 0.450893,
  'Adj Close': 0.360548,
  'Volume': 26432000},
 {'Date': '1980-12-17',
  'Open': 0.462054,
  'High': 0.464286,
  'Low': 0.462054,
  'Close': 0.462054,
  'Adj Close': 0.369472,
  'Volume': 21610400},
 {'Date': '1980-12-18',
  'Open': 0.475446,
  'High': 0.477679,
  'Low': 0.475446,
  'Close': 0.475446,
  'Adj Close': 0.380182,
  'Volume': 18362400},
 {'Date': '1980-12-19',
  'Open': 0.504464,
  'High': 0.506696,
  'Low': 0.504464,
  'Close': 0.504464,
  'Adj Close': 0.403385,
  'Volume': 12157600},
 {'Date': '1980-12-22',
  'Open': 0.529018,
  'High': 0.53125,
  'Low': 0.529018,
  'Cl

In [105]:
select_date = '''
    select `date` from `aapl`
    order by `date` desc
'''

In [ ]:
db2.sql_query(select_date)

[{'date': '2019-06-24'},
 {'date': '2019-06-21'},
 {'date': '2019-06-20'},
 {'date': '2019-06-19'},
 {'date': '2019-06-18'},
 {'date': '2019-06-17'},
 {'date': '2019-06-14'},
 {'date': '2019-06-13'},
 {'date': '2019-06-12'},
 {'date': '2019-06-11'},
 {'date': '2019-06-10'},
 {'date': '2019-06-07'},
 {'date': '2019-06-06'},
 {'date': '2019-06-05'},
 {'date': '2019-06-04'},
 {'date': '2019-06-03'},
 {'date': '2019-05-31'},
 {'date': '2019-05-30'},
 {'date': '2019-05-29'},
 {'date': '2019-05-28'},
 {'date': '2019-05-24'},
 {'date': '2019-05-23'},
 {'date': '2019-05-22'},
 {'date': '2019-05-21'},
 {'date': '2019-05-20'},
 {'date': '2019-05-17'},
 {'date': '2019-05-16'},
 {'date': '2019-05-15'},
 {'date': '2019-05-14'},
 {'date': '2019-05-13'},
 {'date': '2019-05-10'},
 {'date': '2019-05-09'},
 {'date': '2019-05-08'},
 {'date': '2019-05-07'},
 {'date': '2019-05-06'},
 {'date': '2019-05-03'},
 {'date': '2019-05-02'},
 {'date': '2019-05-01'},
 {'date': '2019-04-30'},
 {'date': '2019-04-29'},


In [107]:
# 시스템 환경변수 확인
import sys

In [108]:
sys.path

['c:\\Users\\toqha\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'c:\\Users\\toqha\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'c:\\Users\\toqha\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'c:\\Users\\toqha\\AppData\\Local\\Programs\\Python\\Python39',
 '',
 'C:\\Users\\toqha\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\toqha\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32',
 'C:\\Users\\toqha\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib',
 'C:\\Users\\toqha\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin',
 'c:\\Users\\toqha\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages']

In [ ]:
sys.path[-1].replace('\\', '/')
# 이 경로에 database.py 파일 복붙 -> 이 폴더 위치가 아니더라도 해당 모듈에 접근 가능

'c:/Users/toqha/AppData/Local/Programs/Python/Python39/lib/site-packages'

In [110]:
import database